### Script to plot spatial fields of the regridded PV budget ###

#### *Figs 7, 9-10 in JAMES paper*
---

Manda Chasteen<br>
April 2024

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# My colormaps file:
import custom_colormaps as cc

np.set_printoptions(suppress=True)

import warnings
warnings.simplefilter("ignore") 

In [2]:
# Specify vertical levels to plot:

lev = 31
level = str(lev+1) 

#### Instantaneous PV budget (Fig. 7): ####

In [ ]:
# Tendencies for a given time step are written out at the end of the timestep, so for the timestep from 03:00:18Z - 03:00:36Z, we only need 
# the latter file. 

parent_dir = ''     # file path here for regridded files 

precision = 'single'

file = xr.open_dataset(parent_dir+'pvbudget15to3km_'+precision+'_zenodo_20190520_03.00.36.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=lev).squeeze()

In [ ]:
# Either extract model time step explicltly from file (not included in regridded file attributes by default, so need to obtain it from native file) 
# or set to known value

timestep_dir = ''     # file path here for native mesh files 
dt = xr.open_dataset(timestep_dir+'pvbudget15to3.2019-05-20_03.00.36_ddzold.nc').attrs['config_dt']
# dt = 18.0

# Set time of files 
tdelt = pd.Timedelta(dt, 's')
t_end = pd.Timestamp(2019, 5, 20, 3, 0, 36) 
t_start = t_end - tdelt

In [ ]:
# Read in fields and calculate instantaneous PV budget in units of PVU

pv1 = file.ertel_pv_prev        # PV at beginning of time step
pv2 = file.ertel_pv             # PV at end of time step (i.e., time file is output)

tend_dyn = file.depv_dt_dyn     # dynamics PV tendency over previous time step in PVU/s
tend_diab = file.depv_dt_diab   # diabatic PV tendency over previous time step in PVU/s
tend_fric = file.depv_dt_fric   # frictional PV tendency over previous time step in PVU/s

# Calculate budget terms:
dpv = pv2 - pv1                                         # Change in PV over time step (PVU)

tend_total = (tend_dyn + tend_diab + tend_fric) * dt    # Sum of PV tendencies * time step = net PV change from tendency terms (PVU)
pv2_calc = pv1 + tend_total                             # Calculated PV at time level 2 = PV at time 1 + net PV change from tendency terms
pv_resid = pv2 - pv2_calc                               # Budget residual: actual PV at time level 2 - calculated PV at time level 2 

In [ ]:
# Calculate the max and mean absolute values of the PV change and PV budget residual. Note: the values in the paper were computed using the native files

print('Model precision is: ',precision)
print(r'Maximum absolute PV change over dt: ',(np.abs(dpv)).max().values,' PVU')
print(r'Mean absolute PV change over dt: ',(np.abs(dpv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual: ',(np.abs(pv_resid)).max().values,' PVU')
print(r'Mean absolute PV budget residual: ',(np.abs(pv_resid)).mean().values,' PVU')

In [ ]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file.longitude[file.longitude > lon_central+180.0]
lons_keep = file.longitude[file.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure plotting arrays:
dpv_transposed = dpv.sel(longitude=lons_transposed)
dpv_keep = dpv.sel(longitude=lons_keep)
dpv_new = xr.concat([dpv_transposed, dpv_keep], dim='longitude')

tend_transposed = tend_total.sel(longitude=lons_transposed)
tend_keep = tend_total.sel(longitude=lons_keep)
tend_new = xr.concat([tend_transposed, tend_keep], dim='longitude')

resid_transposed = pv_resid.sel(longitude=lons_transposed)
resid_keep = pv_resid.sel(longitude=lons_keep)
resid_new = xr.concat([resid_transposed, resid_keep], dim='longitude')

In [ ]:
# Makes 3 panel plot of PV budget for single precision simulation:

if 'single' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:
    
    # PV change over dt:
    dpv_plot = ax[0].pcolormesh(lons_new, file.latitude, dpv_new*1e3, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb = fig.colorbar(dpv_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-3}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    
    ax[0].set_title('Potential Vorticity Change Over '+r'$\Delta t$ $-$ Model Level '+level, fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Single Precision', fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Sum of tendencies * dt:
    tend_plot = ax[1].pcolormesh(lons_new, file.latitude, tend_new*1e3, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(tend_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[1].text(0.87, -0.05, r'[$\times\ 10^{-3}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    
    ax[1].set_title(r'($\dot{\sf{q}}_{\sf{dynamics}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{diabatic}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{friction}})$'+r' $ \times\ \Delta t$', 
                    fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left',zorder=12)


    # PV budget residual:
    resid_plot = ax[2].pcolormesh(lons_new, file.latitude, resid_new*1e5, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb2 = fig.colorbar(resid_plot, extend='both',orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Instantaneous PV Budget Residual',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pvbgtpaper_instbgt_pcolormesh_'+precision+'_level'+level+'_03.00.18.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()
 

In [ ]:
# Makes 3 panel plot of PV budget for double precision simulation:

if 'double' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:
    
    # PV change over dt:
    dpv_plot = ax[0].pcolormesh(lons_new, file.latitude, dpv_new*1e3, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb = fig.colorbar(dpv_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-3}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    
    ax[0].set_title('Potential Vorticity Change Over '+r'$\Delta t$ $-$ Model Level '+level, fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Double Precision', fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Sum of tendencies * dt:
    tend_plot = ax[1].pcolormesh(lons_new, file.latitude, tend_new*1e3, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(tend_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[1].text(0.87, -0.05, r'[$\times\ 10^{-3}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    
    ax[1].set_title(r'($\dot{\sf{q}}_{\sf{dynamics}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{diabatic}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{friction}})$'+r' $ \times\ \Delta t$', 
                    fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left',zorder=12)


    # PV budget residual:
    resid_plot = ax[2].pcolormesh(lons_new, file.latitude, resid_new*1e14, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb2 = fig.colorbar(resid_plot, extend='both',orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-14}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Instantaneous PV Budget Residual',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pvbgtpaper_instbgt_pcolormesh_'+precision+'_level'+level+'_03.00.18.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()
 

#### Accumulated PV budget (Fig. 9) ####

In [ ]:
# For the accumulated budget, we need two files.

parent_dir = '/glade/derecho/scratch/chasteen/MPAS_pvpaper/convert_mpas/'

precision = 'single'

file1 = xr.open_dataset(parent_dir+'pvbudget15to3km_'+precision+'_zenodo_20190520_04.00.00.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=lev).squeeze()
file2 = xr.open_dataset(parent_dir+'pvbudget15to3km_'+precision+'_zenodo_20190520_05.00.00.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=lev).squeeze()

In [ ]:
# Either extract model time step explicltly from file (not included in regridded file attributes by default, so need to obtain it from native file) 
# or set to known value

timestep_dir = '/glade/derecho/scratch/chasteen/MPAS_pvpaper/MPAS_fixed_single/'
dt = xr.open_dataset(timestep_dir+'pvbudget15to3.2019-05-20_03.00.36_ddzold.nc').attrs['config_dt']
# dt = 18.0

# Set time of files 
tdelt = pd.Timedelta(1, 'h')
t_end = pd.Timestamp(2019, 5, 20, 5, 0, 00) 
t_start = t_end - tdelt

In [ ]:
# Read in fields and calculate accumulated PV budget in units of PVU
# Note: this workflow can also be applied to calculate the instantaneous budget

pv1 = file1.ertel_pv             # PV at time level 1
pv2 = file2.ertel_pv             # PV at time level 2

# Only want tendencies during period of interest. They continuously accumulate (unless model is restarted), 
# so to obtain the tendencies over the period, you need to difference the accumulated tendency variables
# from each time
tend_dyn = (file2.acc_depv_dt_dyn - file1.acc_depv_dt_dyn)        # dynamics PV tendency over period of interest (here, 1 h)
tend_diab = (file2.acc_depv_dt_diab - file1.acc_depv_dt_diab)     # diabatic PV tendency over period of interest (here, 1 h)
tend_fric = (file2.acc_depv_dt_fric - file1.acc_depv_dt_fric)     # frictional PV tendency over period of interest (here, 1 h)

# Calculate budget terms:
dpv = pv2 - pv1                                         # Change in PV over period (PVU)

tend_total = (tend_dyn + tend_diab + tend_fric) * dt    # Sum of PV tendencies * time step = net PV change from accumulated tendency terms (PVU) 
pv2_calc = pv1 + tend_total                             # Calculated PV at time level 2 = PV at time 1 + net PV change from accumulated tendency terms
pv_resid = pv2 - pv2_calc                               # Budget residual: actual PV at time level 2 - calculated PV at time level 2 

In [ ]:
# Calculate the max and mean absolute values of the PV change and PV budget residual. Note: the values in the paper were computed using the native files

print('Model precision is: ',precision)
print(r'Maximum absolute PV change over period: ',(np.abs(dpv)).max().values,' PVU')
print(r'Mean absolute PV change over period: ',(np.abs(dpv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual: ',(np.abs(pv_resid)).max().values,' PVU')
print(r'Mean absolute PV budget residual: ',(np.abs(pv_resid)).mean().values,' PVU')

In [ ]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file1.longitude[file1.longitude > lon_central+180.0]
lons_keep = file1.longitude[file1.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure plotting arrays:
dpv_transposed = dpv.sel(longitude=lons_transposed)
dpv_keep = dpv.sel(longitude=lons_keep)
dpv_new = xr.concat([dpv_transposed, dpv_keep], dim='longitude')

tend_transposed = tend_total.sel(longitude=lons_transposed)
tend_keep = tend_total.sel(longitude=lons_keep)
tend_new = xr.concat([tend_transposed, tend_keep], dim='longitude')

resid_transposed = pv_resid.sel(longitude=lons_transposed)
resid_keep = pv_resid.sel(longitude=lons_keep)
resid_new = xr.concat([resid_transposed, resid_keep], dim='longitude')

In [ ]:
# Makes 3 panel plot of accumulated PV budget for single precision simulation:

if 'single' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:
    
    # PV change over dt:
    dpv_plot = ax[0].pcolormesh(lons_new, file1.latitude, dpv_new*1e1, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb = fig.colorbar(dpv_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-1}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    
    ax[0].set_title('Potential Vorticity Change Over 1 Hour $-$ Model Level '+level, fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Single Precision', fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Sum of tendencies * dt:
    tend_plot = ax[1].pcolormesh(lons_new, file1.latitude, tend_new*1e1, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(tend_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[1].text(0.87, -0.05, r'[$\times\ 10^{-1}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    
    ax[1].set_title(r'($\Sigma\ \dot{\sf{q}}_{\sf{dynamics}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{diabatic}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{friction}})$'+r' $ \times\ \Delta t$', 
                    fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left',zorder=12)


    # PV budget residual:
    resid_plot = ax[2].pcolormesh(lons_new, file1.latitude, resid_new*1e4, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb2 = fig.colorbar(resid_plot, extend='both',orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-4}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Accumulated PV Budget Residual',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pvbgtpaper_accbgt_'+precision+'_level'+level+'_04Z_05Z.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()
 

In [ ]:
# Makes 3 panel plot of accumulated PV budget for double precision simulation:

if 'double' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:
    
    # PV change over dt:
    dpv_plot = ax[0].pcolormesh(lons_new, file1.latitude, dpv_new*1e1, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb = fig.colorbar(dpv_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-1}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    
    ax[0].set_title('Potential Vorticity Change Over 1 Hour $-$ Model Level '+level, fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Double Precision', fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Sum of tendencies * dt:
    tend_plot = ax[1].pcolormesh(lons_new, file1.latitude, tend_new*1e1, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(tend_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[1].text(0.87, -0.05, r'[$\times\ 10^{-1}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    
    ax[1].set_title(r'($\Sigma\ \dot{\sf{q}}_{\sf{dynamics}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{diabatic}} \:+\: \Delta \, \dot{\sf{q}}_{\sf{friction}})$'+r' $ \times\ \Delta t$', 
                    fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left',zorder=12)


    # PV budget residual:
    resid_plot = ax[2].pcolormesh(lons_new, file1.latitude, resid_new*1e13, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
    cb2 = fig.colorbar(resid_plot, extend='both',orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-13}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Accumulated PV Budget Residual',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pvbgtpaper_accbgt_'+precision+'_level'+level+'_04Z_05Z.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()
 

#### Residual fields from different methods of calculating the instantaneous and accumulated budgets (Fig. 10): ####

*This compares the PV tendency, parent tendency, and exact methods of calculating the accumulated budget.*

*Important note: the parent and exact method budgets should not be directly evaluated with regridded model fields! For the purposes of plotting, 
the budgets were calculated using the native mesh files and then the summed tendencies and residual fields were regridded. Although the regridded
residual field is more precise than the residual field derived from the regridded PV and summed tendency fields, we are using the latter approach
to be consistent with how the PV tendency method's residudal plots were created. The differences are small, but overall, regridding the residual
field itself would be a more precise representation.*

In [7]:
timestep_dir = ''     # file path here for native mesh files 

dt_timestep = xr.open_dataset(timestep_dir+'pvbudget15to3.2019-05-20_03.00.36_ddzold.nc').attrs['config_dt']

# Also need to define the period of evaluation for the accumulated budget
nhours = 1.0
dt_period = nhours * 3600.      # length of period in seconds 

##### Instantaneous budget: #####

In [ ]:
parent_dir = ''     # file path here for regridded files 

precision = 'double'

file1 = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_zenodo_20190520_03.00.18.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=[lev]).squeeze()
file2 = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_zenodo_20190520_03.00.36.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=[lev]).squeeze()
file_bgt = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_resids_1dt_030018_030036_regrid.nc', chunks={'longitude' : 100, 'latitude' : 100}).squeeze()

*PV tendency method:*

In [ ]:
# Read in fields and calculate instantaneous PV budget in units of PVU

pv1 = file1.ertel_pv             # PV at beginning of time step
pv2 = file2.ertel_pv             # PV at end of time step (i.e., time file is output)

tend_dyn = file2.depv_dt_dyn     # dynamics PV tendency over previous time step in PVU/s
tend_diab = file2.depv_dt_diab   # diabatic PV tendency over previous time step in PVU/s
tend_fric = file2.depv_dt_fric   # frictional PV tendency over previous time step in PVU/s

# Calculate budget terms:
dpv = pv2 - pv1                                                     # Change in PV over time step (PVU)

tend_total_pv = (tend_dyn + tend_diab + tend_fric) * dt_timestep    # Sum of PV tendencies * time step = net PV change from tendency terms (PVU)
pv2_calc_pv = pv1 + tend_total_pv                                   # Calculated PV at time level 2 = PV at time 1 + net PV change from tendency terms
pv_resid_pv = pv2 - pv2_calc_pv                                     # Budget residual: actual PV at time level 2 - calculated PV at time level 2 

*Parent tendency method:*

In [ ]:
# Import parent tendency variables:

tend_total_par = file_bgt.rhs_parent_tends
pv_resid_par = dpv - tend_total_par

*Exact method:*

In [ ]:
# Import exact method variables:

tend_total_ex = file_bgt.rhs_exact
pv_resid_ex = dpv - tend_total_ex

In [ ]:
# Calculate the max and mean absolute values of the PV change and PV budget residuals. Note: the values in the paper were computed using the native files

print('Model precision is: ',precision)
print(r'Maximum absolute PV change over period: ',(np.abs(dpv)).max().values,' PVU')
print(r'Mean absolute PV change over period: ',(np.abs(dpv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - PV tendency method: ',(np.abs(pv_resid_pv)).max().values,' PVU')
print(r'Mean absolute PV budget residual - PV tendency method: ',(np.abs(pv_resid_pv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - parent tendency method: ',(np.abs(pv_resid_par)).max().values,' PVU')
print(r'Mean absolute PV budget residual - parent tendency method: ',(np.abs(pv_resid_par)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - exact method: ',(np.abs(pv_resid_ex)).max().values,' PVU')
print(r'Mean absolute PV budget residual - exact method: ',(np.abs(pv_resid_ex)).mean().values,' PVU')

In [ ]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file1.longitude[file1.longitude > lon_central+180.0]
lons_keep = file1.longitude[file1.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure PV tendency method
pv_resid_transposed = pv_resid_pv.sel(longitude=lons_transposed)
pv_resid_keep = pv_resid_pv.sel(longitude=lons_keep)
pv_resid_new = xr.concat([pv_resid_transposed, pv_resid_keep], dim='longitude')

# parent tendency
par_resid_transposed = pv_resid_par.sel(longitude=lons_transposed)
par_resid_keep = pv_resid_par.sel(longitude=lons_keep)
par_resid_new = xr.concat([par_resid_transposed, par_resid_keep], dim='longitude')

# exact tendency
ex_resid_transposed = pv_resid_ex.sel(longitude=lons_transposed)
ex_resid_keep = pv_resid_ex.sel(longitude=lons_keep)
ex_resid_new = xr.concat([ex_resid_transposed, ex_resid_keep], dim='longitude')

In [ ]:
# Makes 3 panel plot of accumulated PV budget for double precision simulation:

if 'double' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:

    # PV method:
    resid_plot = ax[0].pcolormesh(lons_new, file1.latitude, pv_resid_new*1e14, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(),zorder=0)  
    cb0 = fig.colorbar(resid_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb0.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-14}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 

    ax[0].set_title(r'Instantaneous PV Budget Residual $-$ PV Tendency', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Parent method:
    resid_plot2 = ax[1].pcolormesh(lons_new, file1.latitude, par_resid_new*1e14, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(resid_plot2, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-14}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 

    ax[1].set_title(r'Instantaneous PV Budget Residual $-$ Parent Tendency',fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Exact method:
    resid_plot3 = ax[2].pcolormesh(lons_new, file1.latitude, ex_resid_new*1e16, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb2 = fig.colorbar(resid_plot3, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-16}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Instantaneous PV Budget Residual $-$ Exact', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    figname = 'pvbgtpaper_instbgt_methods_'+precision+'_level'+level+'_03.00.18.pdf'
    #fig.savefig(figname,bbox_inches="tight")
    print('saved')
    plt.show()
    

##### Accumulated Budget: #####

In [3]:
parent_dir = ''     # file path here for regridded files 

precision = 'double'

file1 = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_zenodo_20190520_04.00.00.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=[lev]).squeeze()
file2 = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_zenodo_20190520_05.00.00.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).sel(nVertLevels=[lev]).squeeze()
file_bgt = xr.open_dataset(parent_dir+'/pvbudget15to3km_'+precision+'_resids_1h_040000_050000_regrid.nc', chunks={'longitude' : 100, 'latitude' : 100}).squeeze()

*PV tendency method:*

In [11]:
# Evaluate PV tendency method (in this case, we use difference in accumulated tendencies, although it makes no difference overall):

pv1 = file1.ertel_pv             # PV at time level 1
pv2 = file2.ertel_pv             # PV at time level 2

# Only want tendencies during period of interest. They continuously accumulate (unless model is restarted), 
# so to obtain the tendencies over the period, you need to difference the accumulated tendency variables
# from each time
tend_dyn = (file2.acc_depv_dt_dyn - file1.acc_depv_dt_dyn)        # dynamics PV tendency over period of interest (here, 1 h)
tend_diab = (file2.acc_depv_dt_diab - file1.acc_depv_dt_diab)     # diabatic PV tendency over period of interest (here, 1 h)
tend_fric = (file2.acc_depv_dt_fric - file1.acc_depv_dt_fric)     # frictional PV tendency over period of interest (here, 1 h)

# Calculate budget terms:
dpv = pv2 - pv1                                                     # Change in PV over period (PVU)

tend_total_pv = (tend_dyn + tend_diab + tend_fric) * dt_timestep    # Sum of PV tendencies * time step = net PV change from accumulated tendency terms (PVU) 
pv2_calc_pv = pv1 + tend_total_pv                                   # Calculated PV at time level 2 = PV at time 1 + net PV change from accumulated tendency terms
pv_resid_pv = pv2 - pv2_calc                                        # Budget residual: actual PV at time level 2 - calculated PV at time level 2 

*Parent tendency method:*

In [6]:
# Import parent tendency variables:

tend_total_par = file_bgt.rhs_parent_tends
pv_resid_par = dpv - tend_total_par

*Exact method:*

In [8]:
# Import exact method variables:

tend_total_ex = file_bgt.rhs_exact
pv_resid_ex = dpv - tend_total_ex

In [12]:
# Calculate the max and mean absolute values of the PV change and PV budget residuals. Note: the values in the paper were computed using the native files

print('Model precision is: ',precision)
print(r'Maximum absolute PV change over period: ',(np.abs(dpv)).max().values,' PVU')
print(r'Mean absolute PV change over period: ',(np.abs(dpv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - PV tendency method: ',(np.abs(pv_resid_pv)).max().values,' PVU')
print(r'Mean absolute PV budget residual - PV tendency method: ',(np.abs(pv_resid_pv)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - parent tendency method: ',(np.abs(pv_resid_par)).max().values,' PVU')
print(r'Mean absolute PV budget residual - parent tendency method: ',(np.abs(pv_resid_par)).mean().values,' PVU')
print(' -- ')
print(r'Maximum absolute PV budget residual - exact method: ',(np.abs(pv_resid_ex)).max().values,' PVU')
print(r'Mean absolute PV budget residual - exact method: ',(np.abs(pv_resid_ex)).mean().values,' PVU')

Model precision is:  double
Maximum absolute PV change over period:  60.339508409397084  PVU
Mean absolute PV change over period:  0.1957843136210178  PVU
 -- 
Maximum absolute PV budget residual - PV tendency method:  7.973177673648024e-12  PVU
Mean absolute PV budget residual - PV tendency method:  1.0539659120842693e-13  PVU
 -- 
Maximum absolute PV budget residual - parent tendency method:  8.5060847254681e-12  PVU
Mean absolute PV budget residual - parent tendency method:  1.0494126927378156e-13  PVU
 -- 
Maximum absolute PV budget residual - exact method:  1.4210854715202004e-14  PVU
Mean absolute PV budget residual - exact method:  2.8583048378933405e-16  PVU


In [13]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file1.longitude[file1.longitude > lon_central+180.0]
lons_keep = file1.longitude[file1.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure PV tendency method
pv_resid_transposed = pv_resid_pv.sel(longitude=lons_transposed)
pv_resid_keep = pv_resid_pv.sel(longitude=lons_keep)
pv_resid_new = xr.concat([pv_resid_transposed, pv_resid_keep], dim='longitude')

# parent tendency
par_resid_transposed = pv_resid_par.sel(longitude=lons_transposed)
par_resid_keep = pv_resid_par.sel(longitude=lons_keep)
par_resid_new = xr.concat([par_resid_transposed, par_resid_keep], dim='longitude')

# exact tendency
ex_resid_transposed = pv_resid_ex.sel(longitude=lons_transposed)
ex_resid_keep = pv_resid_ex.sel(longitude=lons_keep)
ex_resid_new = xr.concat([ex_resid_transposed, ex_resid_keep], dim='longitude')

In [ ]:
# Makes 3 panel plot of accumulated PV budget for double precision simulation:

if 'double' in precision:

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:

    # PV method:
    resid_plot = ax[0].pcolormesh(lons_new, file1.latitude, pv_resid_new*1e13, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(),zorder=0)  
    cb0 = fig.colorbar(resid_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
    cb0.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-13}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 

    ax[0].set_title(r'Accumulated PV Budget Residual $-$ PV Tendency',fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Parent method:
    resid_plot2 = ax[1].pcolormesh(lons_new, file1.latitude, par_resid_new*1e13, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(),zorder=0)  
    cb1 = fig.colorbar(resid_plot2, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
    cb1.ax.tick_params(labelsize=14) 
    ax[0].text(0.87, -0.05, r'[$\times\ 10^{-13}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 

    ax[1].set_title(r'Accumulated PV Budget Residual $-$ Parent Tendency', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Exact method:
    resid_plot3 = ax[2].pcolormesh(lons_new, file1.latitude, ex_resid_new*1e16, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(), shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
    cb2 = fig.colorbar(resid_plot3, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
    cb2.ax.tick_params(labelsize=14) 
    ax[2].text(0.87, -0.05, r'[$\times\ 10^{-16}$ PVU]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Accumulated PV Budget Residual $-$ Exact', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    figname = 'pvbgtpaper_accubgt_methods_'+precision+'_level'+level+'_04Z_05Z.pdf'
    #fig.savefig(figname,bbox_inches="tight")
    print('saved')
    plt.show()
    